In [ ]:
# Import required libraries
import numpy as np
import sklearn
import pandas as pd
%matplotlib inline

In [ ]:
# Open CSV file of the data
data = pd.read_csv('../data/normalized_trainingdata.csv')

In [ ]:
# Explore data
data.head()

In [ ]:
# Convert target value to categorical data
diagnosis_cat = data.diagnosis.astype('category')
print(diagnosis_cat.cat.category)

# Change to data.diagnosis_cat.cat.codes?

In [ ]:
# Import LDA from sklearn
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, classification_report, precision_score

# Info: http://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html#sklearn.discriminant_analysis.LinearDiscriminantAnalysis
# First paper on using LDA in biology: http://www.jstor.org/stable/2983775?seq=1#page_scan_tab_contents

In [ ]:
# Define model
lda = LinearDiscriminantAnalysis() #default solver is single value decomposition, no shrinkage

y_train = data.diagnosis_cat.cat.codes
x_train = data.drop(['diagnosis'], axis=1)

model = lda.fit(x_train, y_train)
print(model.priors_)
print(model.means_)
print(model.coef_)

In [ ]:
# Make PCDA - principcal component discriminant analysis which is more robust

In [ ]:
# Perform cross validation on PCA to determine number of PC's

In [ ]:
# Perform LDA on PC's - output is discriminant vector with length of number of PC's